# Creating Consensus Haplotypes

Before running the imputation pipeline, it may be necessary to consolidate the haplotypes loaded into the PHG into a set of consensus haplotypes. This notebook walks through the steps necessary to create this consensus set.

## Why do I need consensus haplotypes?

Consensus haplotypes are useful for reducing memory costs when aligning reads to the pangenome. If a PHG contains hundreds of taxa (and therefore may have hundreds of haplotypes for each reference range), the memory required to align reads to it can be prohibitive. RunHapConsensusPipelinePlugin collapses similar haplotypes (e.g. highly conserved sequences, or sequences from closely related taxa) to reduce the overall number of haplotypes per reference range. The degree of collapse can be adjusted based on user-provided parameters (see below).

Sometimes, consensus haplotypes aren't necessary. If you're using only a few taxa from the PHG for imputation (for example, using only the founders in a breeding program), your pangenome may already be small enough to align to successfully. Additionally, the kmer-based read mapping method uses less memory overall than the alignment based read mapping method (using minimap2 or another read alignment tool), so if you intend to use the former for imputation consensus isn't usually necessary. 

## How it works

## Requirements

- A practical haplotype graph database with gvcfs loaded. See the wiki Steps 1 and 2 for setting up your database
- A reference fasta file. Use the same reference as was used to build your database.
- A config file. This should contain the database connection information (See X), and may optionally also contain the parameters for this and other TASSEL plugins.
- A ranking file. This file determines priority when choosing a representative haplotype for a cluster. It has no header and two tab-delimited columns. The first column contains taxa names. The second column contains a score for each taxon. The haplotypes from taxa with higher numbers will be chosen as representatives. In order to prevent ties, each taxon should have a different score.
- A folder containing GVCF files for each taxon 

## Parameters

### Required
- `referenceFasta`: fasta file of the reference genome sequence
- `dbConfigFile`: text file containing the database connection information
- `collapseMethod`: the name of the collapse method to be stored in the database. You will use this method name when you want to map reads or impute paths with this set of consensus haplotypes
- `rankingFile`: text file containing rankings for each taxon. See above for format
- `localGVCFFolder`: path to a local folder containing gvcfs for the taxa that will be used to create the consensus haplotypes

### Optional/Default (Default values in parentheses)
- `collapseMethodDetails`: extra details for the collapse method to be stored in the database
- `minFreq`: (0.5) At each position, if no allele has the minimum frequency, the consensus haplotype allele will be set to missing.
- `maxClusters`: (30) The maximum number of clusters that will be created for a reference range. If mxDiv produces too many clusters, then the smallest divergence that produces maxClusters clusters will be used instead.
- `minSites`: (30) The minimum number of shared sites that can be used to calculate the distance between two taxa
- `minCoverage`: (0.1) For each range, any taxon with proportion of coverage less than this amount will not be used to generate onsensus haplotypes and will not be included in any haplotype group for that range
- `maxThreads`: (1000) The maximum number of threads to be used to create consensi. The actual number of threads will not exceed the number of available CPUs - 2
- `minTaxa`: (1) The minimum number of taxa required in a reference range to create consensus haplotypes for that range.
- `mxDiv`: (0.01) The maximum divergence allowed within clustered haplotypes. Note that maxClusters overrides mxDiv when deciding consensus clusters.
- `clusteringMode`: (upgma_assembly) The clustering method to use. Choose between upgma_assembly and kmer_assembly
- `kmerSize`: (7) When using kmer_assembly mode, the size of the kmers
- `distanceCalculation`: (Euclidian) Distance calculation method. Choose between Euclidian and Manhattan
- `isTestMethod`: (false) Indication if the data is to be loaded against a test method. Data loaded with test methods are not caches with the PHG ktor server
- `clusterStatsFile`: Optional file to print summaries of each reference range's clusters. If no file name is provided, these summaries will not be written.

In [2]:
working_dir = "/workdir/ahb232/phg_sorghum_apr2023/"

DOCKER = "docker1"

DOCKER_VERSION = "biohpc_ahb232/phgdev"

CONFIG = "/phg/config.txt"

METHOD = "consensus_mxDiv_2.5ToNeg4_collapse"

LOCAL_GVCF_FOLDER = "/phg/gvcfRemote/"

LOG_FILE = working_dir + "/logs/create_consensus_mxdiv0025_log.txt"

TO_LOG = ""

if (LOG_FILE != ""):
    TO_LOG = " > " + LOG_FILE
    

In [ ]:
! {DOCKER} run --name create_consensi --rm \
    -v {working_dir}/:/phg/ \
    -t {DOCKER_VERSION} \
    /tassel-5-standalone/run_pipeline.pl -Xmx100G -debug -configParameters {CONFIG} \
    -HaplotypeGraphBuilderPlugin \
    -configFile {CONFIG} \
    -methods {METHOD} \
    -includeVariantContexts true \
    -localGVCFFolder {LOCAL_GVCF_FOLDER} \
    -endPlugin \
    -RunHapConsensusPipelinePlugin \
    -endPlugin {TO_LOG}